In [1]:
import apache_beam as beam

In [2]:
covid_hist_path = ("HIST_PAINEL_COVIDBR_28set2020.csv")
estados_ibge_path = ("EstadosIBGE.csv")

In [3]:
class ToStr(beam.DoFn):
    '''
    Transforma uma tupla de listas em string
    
    Args:
        element (tupple): tupla entre coduf e lista de regiao, governador, uf, estado, casosAcumulado, obitosAcumulado
    Returns:
        strings
    '''
    def process(self, element):
        lista = element[1][0] + element[1][1]
        str1 = ','.join(map(str,lista))
        return [str1]

In [ ]:
class GenerateCovidKey(beam.DoFn):
    '''
    Gera uma tupla de listas a partir dos elementos selecionados
    
    Args:
        element (list): lista =>  [coduf, estado, casosAcumulado, obitosAcumulado]
    
    Returns: 
        tupple => (coduf, estado, casosAcumulado, obitosAcumulado )
    
    '''
    def process(self, element):
        return [(element[0], [element[1], element[2], element[3], element[4]])]

In [ ]:
class GenerateEstadosKey(beam.DoFn):
    '''
    Gera uma tupla de listas a partir dos elementos selecionados
    
    Args:
        element (list): lista =>  [coduf, governador, uf]
    
    Returns: 
        tupple => (coduf, governador, uf)
    
    '''
    
    def process(self, element):
        return [(element[0], [element[1], element[2]])]

In [ ]:
class SelectEstadosFields(beam.DoFn):
    '''
    Seleciona dentre todas as features extraídas, código, governador, uf
    
    Args:
        element(list): lista => [...] todas as features
    
    Returns:
        lista => [coduf, governador, uf]
    
    '''
    def process(self, element):
        return [element[1] + ',' + element[0] + ',' + element[3]]

In [ ]:
class SelectCovidFields(beam.DoFn):
    '''
    Seleciona dentre todas as features extraídas, coduf, regiao, estado, casosAcumulado, obitosAcumulado
    
    Args:
        element(list): lista => [...] todas as features
    
    Returns:
        lista => [coduf, regiao, estado, casosAcumulado, obitosAcumulado]
    
    '''
    def process(self, element):
        return [element[3] + ',' + element[0] + ',' + element[1] + ',' + element[10] + ',' + element[12]]

In [ ]:
class SplitRow(beam.DoFn):
    '''
    Transforma dados csv em listas
    
    Args:
        element(str): todas as features
    
    Returns:
        lista => [...] todas as features
    
    '''
    def process(self, element):
        new_element = element.split(';')
        yield new_element